### Apache Spark Basics
---

### Import  Librieates

In [1]:
import findspark
import os
from pyspark.sql  import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col

### Initialize Spark

In [2]:
findspark.init("C:\\Users\S727953\OneDrive\Software\EGS4.3.4\MySpark\spark")

### Setup Enviroment Variable for Hadoop and Java

In [3]:
os.environ['JAVA_HOME'] ='C:\\Users\S727953\OneDrive\Software\EGS4.3.4\jdk\jdk1.8u121'
os.environ['HADOOP_HOME']  = "C:\\Users\S727953\OneDrive\Software\EGS4.3.4\MySpark\hadoop\bin"

### Check the Spark installed directory

In [4]:
print(findspark.find())

C:\Users\S727953\OneDrive\Software\EGS4.3.4\MySpark\spark


### Create Spark Session and Sprk Context

In [5]:
def createSession():
    spark = SparkSession.builder.master("local").appName("Linear Regression Model").config("spark.executor.memory", "1gb") \
   .getOrCreate()
    sc = spark.sparkContext
    return sc

### Load Spark Context and Read Data Files

In [6]:
sc=createSession()
# Load in the data
rdd=sc.textFile('C:\GIT\POCs\ML\Spark_Tutorials\cal_housing\cal_housing.data')
header=sc.textFile('C:\GIT\POCs\ML\Spark_Tutorials\cal_housing/cal_housing.domain')

In [7]:
rdd.take(10)

['-122.230000,37.880000,41.000000,880.000000,129.000000,322.000000,126.000000,8.325200,452600.000000',
 '-122.220000,37.860000,21.000000,7099.000000,1106.000000,2401.000000,1138.000000,8.301400,358500.000000',
 '-122.240000,37.850000,52.000000,1467.000000,190.000000,496.000000,177.000000,7.257400,352100.000000',
 '-122.250000,37.850000,52.000000,1274.000000,235.000000,558.000000,219.000000,5.643100,341300.000000',
 '-122.250000,37.850000,52.000000,1627.000000,280.000000,565.000000,259.000000,3.846200,342200.000000',
 '-122.250000,37.850000,52.000000,919.000000,213.000000,413.000000,193.000000,4.036800,269700.000000',
 '-122.250000,37.840000,52.000000,2535.000000,489.000000,1094.000000,514.000000,3.659100,299200.000000',
 '-122.250000,37.840000,52.000000,3104.000000,687.000000,1157.000000,647.000000,3.120000,241400.000000',
 '-122.260000,37.840000,42.000000,2555.000000,665.000000,1206.000000,595.000000,2.080400,226700.000000',
 '-122.250000,37.840000,52.000000,3549.000000,707.000000,155

In [8]:
# # Split lines on commas
# rdd = rdd.map(lambda line: line.split(","))

In [9]:
rdd.first()

'-122.230000,37.880000,41.000000,880.000000,129.000000,322.000000,126.000000,8.325200,452600.000000'

In [10]:
rdd.top(2)

['-124.350000,40.540000,52.000000,1820.000000,300.000000,806.000000,270.000000,3.014700,94600.000000',
 '-124.300000,41.840000,17.000000,2677.000000,531.000000,1244.000000,456.000000,3.031300,103600.000000']

In [11]:
# header.collect()

### Transform RDD to DataFrame

In [12]:
df=rdd
df.take(1)

['-122.230000,37.880000,41.000000,880.000000,129.000000,322.000000,126.000000,8.325200,452600.000000']

In [13]:
df=rdd.map(lambda  line: line.split(",")).map(lambda line: Row(line[0],
                                                               line[1],
                                                               line[2],
                                                               line[3],
                                                               line[4],
                                                               line[5],
                                                               line[6],
                                                               line[7],
                                                               line[8]))
df=df.toDF(['longitude',
            'latitude',
            'housingMedianAge',
            'totalRooms',
            'totalBedRooms',
            'population',
            'households',
            'medianIncome',
            'medianHouseValue'])

In [14]:
df.show(10)

+-----------+---------+----------------+-----------+-------------+-----------+-----------+------------+----------------+
|  longitude| latitude|housingMedianAge| totalRooms|totalBedRooms| population| households|medianIncome|medianHouseValue|
+-----------+---------+----------------+-----------+-------------+-----------+-----------+------------+----------------+
|-122.230000|37.880000|       41.000000| 880.000000|   129.000000| 322.000000| 126.000000|    8.325200|   452600.000000|
|-122.220000|37.860000|       21.000000|7099.000000|  1106.000000|2401.000000|1138.000000|    8.301400|   358500.000000|
|-122.240000|37.850000|       52.000000|1467.000000|   190.000000| 496.000000| 177.000000|    7.257400|   352100.000000|
|-122.250000|37.850000|       52.000000|1274.000000|   235.000000| 558.000000| 219.000000|    5.643100|   341300.000000|
|-122.250000|37.850000|       52.000000|1627.000000|   280.000000| 565.000000| 259.000000|    3.846200|   342200.000000|
|-122.250000|37.850000|       52

In [15]:
# df.columns
df.printSchema()

root
 |-- longitude: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- housingMedianAge: string (nullable = true)
 |-- totalRooms: string (nullable = true)
 |-- totalBedRooms: string (nullable = true)
 |-- population: string (nullable = true)
 |-- households: string (nullable = true)
 |-- medianIncome: string (nullable = true)
 |-- medianHouseValue: string (nullable = true)



In [16]:
df=df.withColumn("longitude",df["longitude"].cast(FloatType())).withColumn('latitude',df['latitude'].cast(FloatType())) .withColumn("housingMedianAge",df["housingMedianAge"].cast(FloatType())).withColumn("totalRooms", df["totalRooms"].cast(FloatType()))  .withColumn("totalBedRooms", df["totalBedRooms"].cast(FloatType())).withColumn("population", df["population"].cast(FloatType()))  .withColumn("households", df["households"].cast(FloatType())).withColumn("medianIncome", df["medianIncome"].cast(FloatType())) .withColumn("medianHouseValue", df["medianHouseValue"].cast(FloatType()))

In [17]:
df.printSchema()

root
 |-- longitude: float (nullable = true)
 |-- latitude: float (nullable = true)
 |-- housingMedianAge: float (nullable = true)
 |-- totalRooms: float (nullable = true)
 |-- totalBedRooms: float (nullable = true)
 |-- population: float (nullable = true)
 |-- households: float (nullable = true)
 |-- medianIncome: float (nullable = true)
 |-- medianHouseValue: float (nullable = true)



### Write a User Function to convert the data type of DataFrame columns

In [18]:
def convertColumnTyupe(df,colNames,newType):    
    for colName in colNames:
        print(colName)
        df=df.withColumn(colName, df[colName].cast(newType))
    return df

In [19]:
# Assign all column names to `columns`
colNames =df.columns
newType=FloatType()

# Convert DataFrame columns to FloateType()
df=convertColumnTyupe(df, colNames, newType)

longitude
latitude
housingMedianAge
totalRooms
totalBedRooms
population
households
medianIncome
medianHouseValue


In [20]:
df.printSchema()

root
 |-- longitude: float (nullable = true)
 |-- latitude: float (nullable = true)
 |-- housingMedianAge: float (nullable = true)
 |-- totalRooms: float (nullable = true)
 |-- totalBedRooms: float (nullable = true)
 |-- population: float (nullable = true)
 |-- households: float (nullable = true)
 |-- medianIncome: float (nullable = true)
 |-- medianHouseValue: float (nullable = true)



In [21]:
df.select('population','totalRooms').show(5)

+----------+----------+
|population|totalRooms|
+----------+----------+
|     322.0|     880.0|
|    2401.0|    7099.0|
|     496.0|    1467.0|
|     558.0|    1274.0|
|     565.0|    1627.0|
+----------+----------+
only showing top 5 rows



In [22]:
df.groupBy('housingMedianAge').count().sort('housingMedianAge', ascending=True).show()

+----------------+-----+
|housingMedianAge|count|
+----------------+-----+
|             1.0|    4|
|             2.0|   58|
|             3.0|   62|
|             4.0|  191|
|             5.0|  244|
|             6.0|  160|
|             7.0|  175|
|             8.0|  206|
|             9.0|  205|
|            10.0|  264|
|            11.0|  254|
|            12.0|  238|
|            13.0|  302|
|            14.0|  412|
|            15.0|  512|
|            16.0|  771|
|            17.0|  698|
|            18.0|  570|
|            19.0|  502|
|            20.0|  465|
+----------------+-----+
only showing top 20 rows



In [23]:
df.describe().show()

+-------+-------------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+
|summary|          longitude|         latitude|  housingMedianAge|        totalRooms|    totalBedRooms|        population|       households|      medianIncome|  medianHouseValue|
+-------+-------------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+
|  count|              20640|            20640|             20640|             20640|            20640|             20640|            20640|             20640|             20640|
|   mean|-119.56970444871473|35.63186143109965|28.639486434108527|2635.7630813953488|537.8980135658915|1425.4767441860465|499.5396802325581|3.8706710030346416|206855.81690891474|
| stddev|  2.003531742932898|2.135952380602968| 12.58555761211163|2181.6152515827944| 421.247905943133|  

### Adjust the values of `medianHouseValue`

In [24]:
df=df.withColumn('medianHouseValue', col('medianHouseValue')/10000)

In [25]:
df.show()

+---------+--------+----------------+----------+-------------+----------+----------+------------+----------------+
|longitude|latitude|housingMedianAge|totalRooms|totalBedRooms|population|households|medianIncome|medianHouseValue|
+---------+--------+----------------+----------+-------------+----------+----------+------------+----------------+
|  -122.23|   37.88|            41.0|     880.0|        129.0|     322.0|     126.0|      8.3252|           45.26|
|  -122.22|   37.86|            21.0|    7099.0|       1106.0|    2401.0|    1138.0|      8.3014|           35.85|
|  -122.24|   37.85|            52.0|    1467.0|        190.0|     496.0|     177.0|      7.2574|           35.21|
|  -122.25|   37.85|            52.0|    1274.0|        235.0|     558.0|     219.0|      5.6431|           34.13|
|  -122.25|   37.85|            52.0|    1627.0|        280.0|     565.0|     259.0|      3.8462|           34.22|
|  -122.25|   37.85|            52.0|     919.0|        213.0|     413.0|     19

### Feature Engineering

In [26]:
# Divide `totalRooms` by `households`
roomPerHousehold=df.select(col('totalRooms')/col('households'))

In [27]:
# Divide `population` by `households`
populationPerHousehold=df.select(col('population')/col('households'))

In [28]:
# Divide `totalBedRooms` by `totalRooms`
bedroomsPerHousehold=df.select(col('totalBedRooms')/col('totalRooms'))

In [29]:
# Add the new columns to `df`
df=df.withColumn('roomPerHousehold', col('totalRooms')/col('households')).withColumn("populationPerHousehold", col("population")/col("households")).withColumn("bedroomsPerRoom", col("totalBedRooms")/col("totalRooms"))

In [30]:
df.first()

Row(longitude=-122.2300033569336, latitude=37.880001068115234, housingMedianAge=41.0, totalRooms=880.0, totalBedRooms=129.0, population=322.0, households=126.0, medianIncome=8.325200080871582, medianHouseValue=45.26, roomPerHousehold=6.984126984126984, populationPerHousehold=2.5555555555555554, bedroomsPerRoom=0.14659090909090908)